In [0]:

df = spark.read.table("workspace.default.mock_patient")
display(df)


In [0]:
df_clean=df

In [0]:
display(df_clean)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, bround, format_number
df_clean = df_clean.withColumn("Age_Upd",round(col("Age"),2))

In [0]:
df_clean.show(5)

In [0]:
df_order = spark.read.table("workspace.default.mock_patient_orders_data")
display(df_order)
df = spark.read.table("workspace.default.mock_patient")
display(df)

In [0]:
joined_df = df_clean.join(df_order, df_clean.patient_id == df_order.patient_id, "inner")
display(joined_df)

In [0]:
output_df=joined_df.groupBy("department","order_Month").count()
display(output_df)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import date_format,to_date,try_to_date,coalesce,lit,when,desc
#Generate a SQL query to display all patients over the age of 40 who had an order for a procedure in the Radiology department and were discharged with the reason ‘Transferred’. Include the following details: full name, age, birth date (MM/DD), procedure, discharge date (Month D, YYYY), visit type, and inpatient as “YES/NO”. Order the results by procedure name and by descending patient age.

df_filtered_join = joined_df.filter((joined_df.Age_Upd > 40) & (joined_df.department == "Radiology") & (joined_df.reason=="Transferred"))
df_final = df_filtered_join.select("full_name","Age_Upd",date_format("birth_date", "MM/dd").alias("birth_date" ),"procedure",coalesce(date_format(try_to_date("discharged_date", "MM/dd/yy"), "MMMM d, yyyy"),lit("Invalid_Format")).alias("discharged_date"),when(col("InpatientFlag")==1,"YES").otherwise("NO").alias("inpatient")).orderBy("procedure",desc("Age_Upd"))
display(df_final)

Databricks data profile. Run in Databricks to view.

In [0]:
import pyspark
print(pyspark.__version__)

In [0]:
display(df_filtered_join)

In [0]:
from pyspark.sql.functions import count
#display(df_filtered_join)
df_output= joined_df.groupBy("order_Month","department").agg(count("order_id").alias("cnt"))#.selectExpr("department","cnt")
display(df_output)
df_output.write.mode("overwrite").saveAsTable("orders_month")

Databricks visualization. Run in Databricks to view.